# Cover Letter Writer

### Load model

In [1]:
from langchain_openai import ChatOpenAI

BASE_URL = "http://localhost:1234/v1"
MODEL_NAME = "meta-llama-3.1-8b-instruct"


model = ChatOpenAI(base_url=BASE_URL,api_key="xxx",name=MODEL_NAME)

### Research chain

In [2]:
from langchain.prompts import ChatPromptTemplate

messages = [
    (
        "system" , """You're a research agent tasked with gathering information about a given company, 
        exploring its field of activity, its products or services, and its target market. Analyze the company's history, including 
        its creation date, founders and strategic evolution. Identifies the company's major projects, collaborationsand partnerships, 
        as well as any awards it has received. Finally, studies its HR policy, including talent management, corporate culture, recruitment 
        practicesand employee well-being initiatives. You have access to the Tavily search tool to help you in your research."""
     ),
    ("human"," Do this for the company: {company_name}") ,
    ("placeholder", "{agent_scratchpad}")
]
search_prompt_template = ChatPromptTemplate.from_messages(messages)

In [80]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=10)
tools = [tavily_tool]

In [81]:
from langchain.agents import AgentExecutor, create_openai_functions_agent

agent = create_openai_functions_agent(model, tools, search_prompt_template)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False)

In [82]:
from langchain_core.runnables import RunnableLambda

get_output = RunnableLambda(lambda x : x['output'])

agent_chain = agent_executor | get_output

In [83]:
#result = agent_chain.invoke({"company_name": "fieldbox.ai"})
#print(result[:100])

### Write chain

In [3]:
messages = [
    ("system","{prompt}"),
    ("human",""" 
     template : {template}
     profil : {profil} 
     offer_data : {offer} 
     company_data : {company_data}"""),
]
write_prompt_template = ChatPromptTemplate.from_messages(messages)

In [4]:
from utils import load_json,load_txt
import os

DATA_DIR = "../data/"

prompt = load_txt(os.path.join(DATA_DIR,"promptV1.txt"))
offer = ""
profil = load_json(os.path.join(DATA_DIR,"profil.json"))
template = load_txt(os.path.join(DATA_DIR,"templateBrut.txt"))
len(prompt),len(offer),len(profil)

(1151, 0, 3000)

In [5]:
write_prompt_template = write_prompt_template.partial(prompt=prompt,offer=offer,profil=profil,template=template)

In [6]:
from langchain_core.output_parsers import StrOutputParser

write_chain = write_prompt_template | model | StrOutputParser()

In [7]:
result = write_chain.invoke({"company_data":"no data"})
print(result)

BadRequestError: Error code: 400 - {'error': 'The model has crashed without additional information. (Exit code: 139)'}

### Final Chain

In [89]:
final_chain = agent_chain | write_chain

In [90]:
result = final_chain.invoke({"company_name" : "CapGemini"})

In [91]:
print(result)

<think>
Alright, I need to help this user craft a compelling motivation letter for a data scientist role at Capgemini based on the provided template and their profile.

First, let's break down what they're asking for. The user is Theo Castillo, 19 years old from Bordeaux, in his second year of an informatics program with a strong passion for AI. He has experience with Python, Java, HTML, and several libraries like TensorFlow and PyTorch. He's also done a stage abroad, which shows he's adaptable.

He wants the motivation letter to highlight his alignment with Capgemini's values and projects, showcasing how his skills and experiences can contribute to their work in Data and IAM. The template provided should guide me in structuring this effectively.

I need to make sure that Theo emphasizes his passion for AI, his technical expertise, his experience in real-world projects, and his adaptability from his international stage. Also, aligning these points with Capgemini's goals and values will